### Ingest races.csv file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from formula1.formula1_constants import raw_folder_path, processed_folder_path, presentation_folder_path

In [ ]:
from formula1.formula1_utils import add_ingestion_date, re_arrange_partition_column, df_column_to_list

##### Step 1 - Read the CSV file using the spark dataframe reader API

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [ ]:
races_schema = StructType(fields=[StructField("raceId", IntegerType(), False),
                                  StructField("year", IntegerType(), True),
                                  StructField("round", IntegerType(), True),
                                  StructField("circuitId", IntegerType(), True),
                                  StructField("name", StringType(), True),
                                  StructField("date", DateType(), True),
                                  StructField("time", StringType(), True),
                                  StructField("url", StringType(), True) 
])

In [ ]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv(f"{raw_folder_path}/races.csv")

##### Step 2 - Add ingestion date and race_timestamp to the dataframe

In [ ]:
from pyspark.sql.functions import to_timestamp, concat, col, lit

In [ ]:
races_with_timestamp_df = races_df.withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
.withColumn("data_source", lit(v_data_source))

In [ ]:
races_with_ingestion_date_df = add_ingestion_date(races_with_timestamp_df)

##### Step 3 - Select only the columns required & rename as required

In [ ]:
races_selected_df = races_with_ingestion_date_df.select(col('raceId').alias('race_id'), col('year').alias('race_year'), 
                                                   col('round'), col('circuitId').alias('circuit_id'),col('name'), col('ingestion_date'), col('race_timestamp'))

##### Write the output to processed container in parquet format

In [ ]:
races_selected_df.write.mode('overwrite').partitionBy('race_year').parquet(f'{processed_folder_path}/races')